In [32]:
import plotly.graph_objects as go
from plotly import tools
import plotly.offline as py
import plotly.express as px
import pandas as pd

import numpy as np

caps = ["mega","big", "micro","nano", "mid"]
dfs = []
cap_res = []
for cap in caps:
    df = pd.read_csv(f"{cap}_meta_pl.csv", index_col=0)
    
    cap_res.append(df["pl5"].mean())
    df.fillna(0, inplace=True)
    dfs.append(df)
big_df = pd.concat(dfs)
sectors=list(big_df["Industry"].value_counts().keys()[:10])


results =[]
for sector in sectors:
    sector_df = big_df[big_df["Industry"] == sector]
    results.append(sector_df["pl5"].mean())
    
"""print(sectors)
print(results)
print(caps)
print(cap_res)"""

big_df = big_df.sort_values(by=['pl5'])
#print(big_df[["Symbol","pl5"]])
nasdaq_df = pd.read_csv("data/nasdaq_tickers.csv", index_col=0)
nasdaq_df = nasdaq_df.fillna(0)
nasdaq_df["shares_outstanding"] = nasdaq_df["Market Cap"].astype(float)/nasdaq_df["Last Sale"].str[1:].astype(float)

volumes =[]
last_price = []
for symbol in big_df["Symbol"]:
    try:
        stock_df = pd.read_csv(f"data/stock_data/{symbol}.csv",index_col=0)
        volume_traded = stock_df["Volume"].sum()
        volumes.append(volume_traded)
        last_price.append(stock_df["Close"].iloc[-1])
        
    except Exception as e:
        
        volumes.append(0)
        last_price.append(0)
big_df["volume_traded_period"] = volumes
big_df["last_price"] = last_price
big_df["shares_outstanding"] = big_df["Market Cap"].astype(float)/big_df["last_price"].astype(float)
big_df["trade_turnover"] = big_df["volume_traded_period"]/big_df["shares_outstanding"]

#big_df["trade_turnover"] = big_df["volume_traded_period"]

     Symbol    Market Cap                                           Industry  \
1745  GOOGL  1.831042e+12                  Internet and Information Services   
1744   GOOG  1.840707e+12                  Internet and Information Services   
2558    MDB  3.245573e+10   Retail: Computer Software & Peripheral Equipment   
2691   MRNA  1.722561e+11  Biotechnology: Biological Products (No Diagnos...   
2677   MPWR  3.326627e+10                                     Semiconductors   
...     ...           ...                                                ...   
3309   QDEL  6.231383e+09  Biotechnology: In Vitro & In Vivo Diagnostic S...   
2870   NVAX  1.705613e+10  Biotechnology: Biological Products (No Diagnos...   
2042   ILMN  1.382208e+11                               Medical Specialities   
916    COIN  4.935737e+10                                       EDP Services   
245    AMZN  1.705565e+12                     Catalog/Specialty Distribution   

             pl5  volume_traded_period 

In [36]:
import plotly.express as px
fig = px.scatter(x=big_df["trade_turnover"], y=big_df["pl5"])
fig.update_yaxes(type='log')
fig.update_xaxes(type='log')
fig.show()

In [37]:
 big_df =big_df[["Symbol","trade_turnover", "pl5"]] 
json_df = big_df.to_json(orient='index')
with open('trade_turnover.json', 'w') as f:
    f.write(json_df)